# MACS 30100 PS7

## Ling Dai

In [203]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier

# Problem 1

## 1(a)

In [204]:
#import auto data
auto = pd.read_csv('auto.csv', header = 0, sep=",", na_values = "?")
auto.dropna(inplace = True)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [205]:
median_mpg = np.median(auto['mpg'])
print('Median mpg: {}'.format(median_mpg))

def high_value(value, threshold):
    if value >= threshold:
        return 1
    else:
        return 0

auto['mpg_high'] = auto['mpg'].apply(high_value, threshold = median_mpg)

auto.head()

Median mpg: 22.75


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


In [206]:
import statsmodels.api as sm
X = sm.add_constant(auto.iloc[:,1:-2])
y = auto.iloc[:,-1]

In [207]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression as LR


k = 4
MSE_vec = np.zeros(k)
class_0_error_rates = []
class_1_error_rates = []

kf_log = KFold(n_splits=4, shuffle=True, random_state=15)
k_ind = int(0)

for train, test in kf_log.split(X):
    X_train = X.iloc[train]
    y_train = y.iloc[train]
    X_test = X.iloc[test]
    y_test = y.iloc[test]
    reg = LR().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_0_error_rate = cm[1][0]/(cm[0][0]+cm[1][0])
    class_1_error_rate = cm[0][1]/(cm[1][1]+cm[0][1])
    class_0_error_rates.append(class_0_error_rate)
    class_1_error_rates.append(class_1_error_rate)
    MSE_vec[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('\nTest set {}:'.format(k_ind))
    print('Test MSE is {}'.format(MSE_vec[k_ind]))
    print('Class 0 error rate is {}'.format(class_0_error_rate))
    print('Class 1 error rate is {}'.format(class_1_error_rate))
    k_ind += 1

print('\nAverage:')
print('Test MSE is {}'.format(MSE_vec.mean()))   
print('Class 0 error rate is {}'.format(np.mean(class_0_error_rates)))
print('Class 1 error rate is {}'.format(np.mean(class_1_error_rates)))


Test set 0:
Test MSE is 0.09183673469387756
Class 0 error rate is 0.057692307692307696
Class 1 error rate is 0.13043478260869565

Test set 1:
Test MSE is 0.10204081632653061
Class 0 error rate is 0.12244897959183673
Class 1 error rate is 0.08163265306122448

Test set 2:
Test MSE is 0.1326530612244898
Class 0 error rate is 0.15217391304347827
Class 1 error rate is 0.11538461538461539

Test set 3:
Test MSE is 0.10204081632653061
Class 0 error rate is 0.046511627906976744
Class 1 error rate is 0.14545454545454545

Average:
Test MSE is 0.10714285714285715
Class 0 error rate is 0.09470670705864986
Class 1 error rate is 0.11822664912727025


According to the results above, the average test MSE is around 0.1071, the classification error rate for class 0 (mpg_high = 0) is around 0.0947, and that for class 1 (mpg_high = 1) is around 0.1182.

## 1(b)

In [208]:
X_rf = auto[['cylinders', 'displacement', 'horsepower', 'weight',
          'acceleration', 'year', 'origin']].values
y = auto['mpg_high'].values

In [209]:
rfc = RandomForestClassifier(n_estimators=20, max_features=2,
                             bootstrap=True, oob_score=True, random_state = 25)
rfc.fit(X_rf, y)
rf_pred = rfc.oob_decision_function_

rfc_df = pd.DataFrame({'y': y,
                       'oob_class_0_prob':rfc.oob_decision_function_[:,0],
                       'oob_class_1_prob':rfc.oob_decision_function_[:,1]},)

rfc_df['oob_prediction_'] = rfc_df['oob_class_0_prob'].apply(lambda x: 1 if x<0.5 else 0)

rfc_df.head()

,oob_class_0_prob,oob_class_1_prob,y,oob_prediction_
0,1.0,0.0,0,0
1,1.0,0.0,0,0
2,1.0,0.0,0,0
3,1.0,0.0,0,0
4,1.0,0.0,0,0


In [210]:
MSE = len(rfc_df[rfc_df['oob_prediction_']!=rfc_df['y']])/len(rfc_df)
Class_0_error_rate = len(rfc_df[(rfc_df['oob_prediction_']==1)&
                                (rfc_df['y']==0)])/len(rfc_df[rfc_df['y']==0])
Class_1_error_rate = len(rfc_df[(rfc_df['oob_prediction_']==0)&
                                (rfc_df['y']==1)])/len(rfc_df[rfc_df['y']==1])

print('MSE is {}'.format(MSE))
print('Error rate of class 0 is {}'.format(Class_0_error_rate))
print('Error rate of class 1 is {}'.format(Class_1_error_rate))

MSE is 0.0663265306122449
Error rate of class 0 is 0.07142857142857142
Error rate of class 1 is 0.061224489795918366


According to the results above, the average test MSE is around 0.0663, the classification error rate for class 0 (mpg_high = 0) is around 0.0714, and that for class 1 (mpg_high = 1) is around 0.0612.

## 1(c)

In [211]:
import statsmodels.api as sm
X = sm.add_constant(auto.iloc[:,1:-2])
y = auto.iloc[:,-1]
X.head()

,const,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,1.0,8,307.0,130.0,3504,12.0,70,1
1,1.0,8,350.0,165.0,3693,11.5,70,1
2,1.0,8,318.0,150.0,3436,11.0,70,1
3,1.0,8,304.0,150.0,3433,12.0,70,1
4,1.0,8,302.0,140.0,3449,10.5,70,1


In [212]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import confusion_matrix

k = 4
MSE_vec = np.zeros(k)
class_0_error_rates = []
class_1_error_rates = []

kf_log = KFold(n_splits=4, shuffle=True, random_state=15)
k_ind = int(0)

for train, test in kf_log.split(X):
    X_train = X.iloc[train]
    y_train = y.iloc[train]
    X_test = X.iloc[test]
    y_test = y.iloc[test]
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_0_error_rate = cm[1][0]/(cm[0][0]+cm[1][0])
    class_1_error_rate = cm[0][1]/(cm[1][1]+cm[0][1])
    class_0_error_rates.append(class_0_error_rate)
    class_1_error_rates.append(class_1_error_rate)
    MSE_vec[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('\nTest set {}:'.format(k_ind))
    print('Test MSE is {}'.format(MSE_vec[k_ind]))
    print('Class 0 error rate is {}'.format(class_0_error_rate))
    print('Class 1 error rate is {}'.format(class_1_error_rate))
    k_ind += 1

print('\nAverage:')
print('Test MSE is {}'.format(MSE_vec.mean()))   
print('Class 0 error rate is {}'.format(np.mean(class_0_error_rates)))
print('Class 1 error rate is {}'.format(np.mean(class_1_error_rates)))


Test set 0:
Test MSE is 0.08163265306122448
Class 0 error rate is 0.0392156862745098
Class 1 error rate is 0.1276595744680851

Test set 1:
Test MSE is 0.08163265306122448
Class 0 error rate is 0.0851063829787234
Class 1 error rate is 0.0784313725490196

Test set 2:
Test MSE is 0.10204081632653061
Class 0 error rate is 0.09302325581395349
Class 1 error rate is 0.10909090909090909

Test set 3:
Test MSE is 0.10204081632653061
Class 0 error rate is 0.046511627906976744
Class 1 error rate is 0.14545454545454545

Average:
Test MSE is 0.09183673469387754
Class 0 error rate is 0.06596423824354086
Class 1 error rate is 0.11515910039063981


According to the results above, the average test MSE is around 0.0918, the classification error rate for class 0 (mpg_high = 0) is around 0.0660, and that for class 1 (mpg_high = 1) is around 0.1152.

## 1(d)

According to the reported results above, the oob prediction of random forest had the lowest MSE, and the error rates for both classes were relatively low. Therefore, the random forest classifier seems to be the best model among these three. Moreover, the random forest classifier had the lowest error rate for class 1 among all models.

The second most accurate predictive model here is SVM, with an average test MSE of around 0.0918. The class 0 error rate for SVM was around 0.066, which is the lowest among all models. However, its class 1 error rate is around 0.115, which is relatively high.

The least accurate model here is the logistic regression model. Its average test MSE is around 0.107, which is lower than that of SVM by a small margin. Although its error rate of class 1 is relatively close to that of SVM, the error rate of class 0 for logistic regression is much higher than that of SVM, making its overall error rate higher for logistic regression.